In [1]:
import numpy as np
import os
import dv_processing as dv
from datetime import timedelta
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import cv2


import sys
sys.path.append('../')
from src.data.preprocessing import *
from src.data.custom_dataset import *
from src.models.convlstm import ConvLSTM

from src.utils.plotting import *


C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataloader, test_dataloader = create_dataloader(
    data_dir='../data/seq_data',
    batch_size=32,
    num_workers=1,
)

Total number of sequences: 307
Number of training sequences: 245
Number of test sequences: 62
Training DataLoader:
torch.Size([32, 300, 1, 32, 32]) torch.Size([32, 300])
Test DataLoader:
torch.Size([32, 300, 1, 32, 32]) torch.Size([32, 300])
Number of training batches: 8
Number of test batches: 2


## 1. ConvLSTM

In [ ]:
checkpoint_path = '../saved_models/epoch=49-step=400.ckpt'
model = ConvLSTM.load_from_checkpoint(checkpoint_path, in_channels=1, feature_size=128)

: 

In [ ]:
visualize_top_errors(model, test_dataloader, model_name="ConvLSTM", top_n=5, save_path='convlstm_top_errors.mp4')

[(46, 300), (27, 299), (12, 287), (45, 282), (34, 281), (23, 279), (50, 263), (26, 259), (1, 257), (53, 243), (38, 237), (48, 225), (15, 213), (20, 211), (32, 208), (29, 207), (18, 205), (39, 204), (54, 204), (30, 180), (49, 179), (41, 174), (44, 174), (2, 158), (4, 140), (55, 124), (43, 123), (47, 95), (13, 73), (9, 61), (8, 58), (19, 48), (25, 25), (61, 22), (17, 19), (60, 13), (36, 11), (40, 7), (42, 5), (31, 4), (0, 3), (3, 2), (5, 2), (14, 2), (33, 1), (6, 0), (7, 0), (10, 0), (11, 0), (16, 0), (21, 0), (22, 0), (24, 0), (28, 0), (35, 0), (37, 0), (51, 0), (52, 0), (56, 0), (57, 0), (58, 0), (59, 0)]
